
<br>
Analysis of a data set from US dept of labor Bureau of Labor Statistics to reveal undelying connection between gender, race, and physical labor<br>
TLDR;<br>
    - There is an inverse relationship between the percentage representation of women in an industry, and that industries liklihood to be classified as "physical labor"<br>
    - The percentage of Whites, Blacks, Hispanics, Asians, and the total number of people employed in an industry are irrelevant in predicting if the industry will be classified as “physical labor”.<br>
    - Women are more likely to work in industries with larger populations.<br>


In [ ]:
import pandas
from pdb import set_trace as bp
from numpy import log

Stats models function naming

In [ ]:
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from statsmodels.compat import lzip
lm      = smf.ols

In [ ]:
BLS_CSV = "./data/bls/bls.csv"

In [ ]:
def bptest(olsModelData):
    """
    Python re-implementation of R Breusch Pagan test of homoskedasticity 'bptest()'
    @param olsModelData : linear model resulting from statsmodels.formula.api.ols call
    @return res         : Wrapper instance around statsmodels.stats.api.het_breuschpagan call
    """
    # Define toString value for bptest result
    class Bpresult:
        """
        Wrapper class around statsmodels.stats.api.het_breuschpagan
          .fieldnames: String fieldnames of Breusch Pagan test results
          .bpvalues  : Values corresponding to fieldnames
          .result    : Dictionary of combined fieldnames and values
          __str__()  : Overload ToString method for nice printing
        """
        def __init__(self, olsModelData):
            """
            Initialization of Bpresult wrapper
            @param olsModelData: linear model resulting from statsmodels.formula.api.ols call
            """
            self.olsModelData = olsModelData
            self.fieldnames   = ['Lagrange multiplier statistic', 'p-value', 'f-value', 'p-value']
            self.bpvalues     = sms.het_breuschpagan(olsModelData.resid, olsModelData.model.exog)
            self.result = {}
            for i in range(len(self.fieldnames)):
                key   = self.fieldnames[i]
                value = self.bpvalues[i]
                self.result[key] = value
        def __str__(self):
            """
            Bpresult wrapper __str__ override
            Will print model formula, Lagrange multiplier statistic (BP), df_model value, and p-value
            """
            rep = '\n\t-- studentized Breusch-Pagan test --\n'
            rep += f"formula = {self.olsModelData.model.formula}\n"
            rep += f"BP      = {self.result['Lagrange multiplier statistic']}\n"
            rep += f"df      = {self.olsModelData.df_model}\n"
            if (self.result['p-value'] < (2.2 * (10**-16))):
                rep += f"p-value < 2.2e-16\n"
            else:
                rep += f"p-value = {self.result['p-value']}\n"
            rep +='\n'
            return rep
    res = Bpresult(olsModelData)
    return res

phy_lbr:	Boolean is profession "Physical Labor"<br>
tot_emp:	Total employed(Thousands)<br>
women:		Women(%)<br>
white:		White(%)<br>
black:		Black(%)<br>
asian:		Asian(%)<br>
hispanic:	Hispanic(%)

In [ ]:
bls_df = pandas.read_csv(BLS_CSV)
print(bls_df.describe())

M1<br>
Use R code get the OLS estimates of the parameters of the following regression model, where u i is<br>
the regression error:<br>
phy_lbr(i) = β_0 + β_1 ln(tot_emp(i)) + β_2 women(i) + β_3 white(i) + β_4 black(i) + β_5 asian(i) + β_6 hispanic(i) + u(i)

In [ ]:
bls_ols_phylbr = lm(formula = "phy_lbr ~ log(tot_emp) + women + white + black + asian + hispanic", data=bls_df).fit()
print(bls_ols_phylbr.summary())

F test: INSIGNIFICANT VARIABLES: tot_emp white black asian and hispanic<br>
test_1 <- linearHypothesis(bls_ols_phylbr, c("log(tot_emp)=0", "white=0", "black=0", "asian=0", "hispanic=0"))<br>
ODO

Testing homoskedasticity

In [ ]:
print(bptest(bls_ols_phylbr))

T-Test of coefficients: heteroskedasticity-corrected<br>
ODO

M2<br>
the regression error:<br>
women(i) = β_0 + β_1 ln(tot_emp(i)) + β_2 phy_lbr + β_3 white(i) + β_4 black(i) + β_5 asian(i) + β_6 hispanic(i) + u(i)

In [ ]:
bls_ols_women = lm(formula = "women ~ log(tot_emp) + phy_lbr + white + black + asian + hispanic", data=bls_df).fit()
print(bls_ols_women.summary())

F test: asain=1<br>
test_2 <- linearHypothesis(bls_ols_phylbr, c("asian=1"))<br>
ODO

Testing homoskedasticity

In [ ]:
print(bptest(bls_ols_women))

T-Test of coefficients: heteroskedasticity-corrected<br>
ODO

###############################################################################################################<br>
Extra fun stuff :)<br>
###############################################################################################################<br>
M1: Ignoring insignificant variables<br>
Exasterbate M1 findings<br>
ls_ols_phylbr_clean = lm(formula = "phy_lbr ~ women", data=bls_df).fit()<br>
rint(bls_ols_phylbr_clean.summary())

M2: Ignoring insignificant variables<br>
Exasterbate M2 findings<br>
ls_ols_women_clean = lm(formula = "women ~ log(tot_emp) + phy_lbr + white + black + asian", data=bls_df).fit()<br>
rint(bls_ols_phylbr_clean.summary())

Actual numbers of things<br>
women variable is % of workforce that is women<br>
tot_emp is the number of people in the workforce<br>
women * tot_emp is the number of WOMEN in the workforce<br>
ls_ols_num = lm(formula = "I(women*tot_emp) ~ phy_lbr + I(white*tot_emp) + I(black*tot_emp) + I(asian*tot_emp) + I(hispanic*tot_emp)", data = bls_df).fit()<br>
ls_ols_num = lm(formula = "phy_lbr ~ I(women*tot_emp) + I(white*tot_emp) + I(black*tot_emp) + I(asian*tot_emp) + I(hispanic*tot_emp)", data = bls_df).fit()<br>
rint(bls_ols_phylbr_clean.summary())

Percent increase in population as a function of demographics<br>
Which demographics are indicators of labor force growth<br>
ls_ols_totemp = lm(formula = "log(tot_emp) ~ phy_lbr + women + white + black + asian + hispanic", data = bls_df).fit()<br>
rint(bls_ols_totemp.summary())